In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [2]:
#--------------------------------------------------------------
def draw_vector(v0, v1, ax=None):
    ax = ax or plt.gca()
    arrowprops=dict(arrowstyle='->',
                    linewidth=2,
                    shrinkA=0, shrinkB=0)
    ax.annotate('', v1, v0, arrowprops=arrowprops)

#--------------------------------------------------------------
def find_main_component_dft(image, th):
    #
    
    # dft and mag
    dftIm = cv2.dft(np.float32(image),flags = cv2.DFT_COMPLEX_OUTPUT)
    dftImShift = np.fft.fftshift(dftIm)
    magIm = 20*np.log(cv2.magnitude(dftImShift[:,:,0],dftImShift[:,:,1]))
    
    fig = plt.figure(figsize=(20,5))
    
    original = fig.add_subplot(141)
    plt.imshow(image, cmap = 'gray')
    plt.title('Original Image', fontsize=25)
    plt.xlabel('x', fontsize=20)
    plt.ylabel('y', fontsize=20)
    plt.xticks()
    plt.yticks()

    mag = fig.add_subplot(142)
    plt.imshow(magIm, cmap = 'gray')
    plt.title('Magnitude of FT', fontsize=25)
    plt.xlabel('$\omega_x$', fontsize=20)
    plt.ylabel('$\omega_y$', fontsize=20)
    plt.xticks()
    plt.yticks()
    plt.colorbar()
    
    # Threshold
    #selection of magnitudes for biary picture: if the element in magIm >= maxMag/5*th => element = maxMag,
    # else is 0.
    maxMag = np.max(magIm)
    ret, thresh = cv2.threshold(magIm, maxMag/5*th, maxMag, \
                                cv2.THRESH_BINARY)
    # Plot Threshold
    gradient = fig.add_subplot(143)
    plt.imshow(thresh, cmap = 'gray')
    plt.title('Magnitude Binary', fontsize=25)
    plt.xlabel('$\omega_x$', fontsize=20)
    plt.ylabel('$\omega_y$', fontsize=20)
    plt.xticks()
    plt.yticks()
    plt.grid(0)
    
    # Principal Component Analysis
    y,x = np.where(thresh == np.max(thresh))
    y = -y
    X = np.ones((len(x),2))
    X[:,0] = x; X[:,1] = y
    pca = PCA(n_components=2)
    pca.fit(X)
    #print(pca.components_)
    #print(pca.explained_variance_)
    #fig = plt.figure(figsize=(20,5))
    
    vec = fig.add_subplot(144)
    fig_scatter = plt.scatter(x,y)
        
    left  = 0.0  # the left side of the subplots of the figure
    right = 1.5    # the right side of the subplots of the figure
    bottom = 0.1   # the bottom of the subplots of the figure
    top = 1  # the top of the subplots of the figure
    wspace = 0.3   # the amount of width reserved for space between subplots,
               # expressed as a fraction of the average axis width
    hspace = 0.1   # the amount of height reserved for space between subplots,
               # expressed as a fraction of the average axis height
    plt.subplots_adjust(left, bottom, right, top,
                wspace, hspace)
    
    for sigma, direction in zip(pca.explained_variance_, pca.components_):
        v = direction.T * 2 * np.sqrt(sigma)
        zero = [np.mean(x), np.mean(y)]
        draw_vector(zero, zero - v)
    plt.axis('equal');
    
    print('''The angle between the main component and the horizontal is 
        \u03B1 = %2.3f \n''' % (np.arctan(pca.components_[0,1]/pca.components_[0,0])\
         /np.pi/2 * 360))
    print('''The angle between the second component and the horizontal is 
        \u03B2 = %2.3f \n''' % (np.arctan(-pca.components_[1,1]/pca.components_[1,0])\
         /np.pi/2 * 360))
    # pca.explained_variance_[0] = variance along pricipal comp
    # pca.explained_variance_[1] = variance along ortogonal comp
    print("The alignment factor is: %.3g" %  (pca.explained_variance_[1]/pca.explained_variance_[0]))
    
    plt.show()
    